In [15]:
import numpy as np
from numpy import ndarray

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Lambda
from torchvision.models import vgg16_bn, VGG16_BN_Weights

from utils_numpy import collect_augment_aggregate_data, create_batch_data

In [16]:
num_of_clients = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
transform = Compose([
    ToTensor(),
])

target_transform = Compose([
    Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), 1))
])

# load data
train_dataset = CIFAR10('./data', train=True, transform=transform, target_transform=target_transform,
                        download=True)
train_loader = DataLoader(train_dataset, batch_size=train_dataset.data.shape[0], shuffle=False)
test_dataset = CIFAR10('./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=test_dataset.data.shape[0], shuffle=False)


all_train_data, all_train_labels = collect_augment_aggregate_data(train_dataset, train_loader, num_of_clients, 10)
all_test_data, all_test_labels = collect_augment_aggregate_data(test_dataset, test_loader, num_of_clients, 10)
all_train_data, all_train_labels, all_test_data, all_test_labels = create_batch_data(all_train_data, all_train_labels, all_test_data, all_test_labels, 256)

vgg_backbone = vgg16_bn(weights=VGG16_BN_Weights.DEFAULT).eval()
vgg_backbone = torch.nn.Sequential(*(list(vgg_backbone.children())[:-1])).to(device)


with torch.no_grad():
    train_data_all, train_label_all, test_data_all, test_label_all = [], [], [], []
    for train_data, train_label in zip(all_train_data, all_train_labels):
        train_data = torch.tensor(train_data)
        train_data = train_data.to(device)
        train_data = vgg_backbone(train_data).reshape(train_data.size(0), -1).to('cpu').numpy()

        train_data_all.append(train_data)
        train_label_all.append(train_label)

    for test_data, test_label in zip(all_test_data, all_test_labels):
        test_label_all.append(test_label)
        test_data = torch.tensor(test_data)
        test_data = test_data.to(device)
        test_data = vgg_backbone(test_data).reshape(test_data.size(0), -1).to('cpu').numpy()
        test_data_all.append(test_data)



train_data_all, train_label_all = np.concatenate(train_data_all, axis=0), np.concatenate(train_label_all, axis=0)
test_data_all, test_label_all = np.concatenate(test_data_all, axis=0), np.concatenate(test_label_all, axis=0)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
import os
root_path = './data/cifar10-vgg-64/'
if not os.path.exists(root_path):
    os.makedirs(root_path)

In [18]:
with open(os.path.join(root_path, 'train_data.npy'), 'wb') as fp:
    np.save(fp, train_data_all)

with open(os.path.join(root_path, 'train_label.npy'), 'wb') as fp:
    np.save(fp, train_label_all)

with open(os.path.join(root_path, 'test_data.npy'), 'wb') as fp:
    np.save(fp, test_data_all)

with open(os.path.join(root_path, 'test_label.npy'), 'wb') as fp:
    np.save(fp, test_label_all)

In [19]:
with open(os.path.join(root_path, 'train_data.npy'), 'rb') as fp:
    train_data_all = np.load(fp)

with open(os.path.join(root_path, 'train_label.npy'), 'rb') as fp:
    train_label_all = np.load(fp)

with open(os.path.join(root_path, 'test_data.npy'), 'rb') as fp:
    test_data_all = np.load(fp)

with open(os.path.join(root_path, 'test_label.npy'), 'rb') as fp:
    test_label_all = np.load(fp)